# Dataset Creation - TRsim

## Step 1: Create the foundation files

Within the folder data/scene_datasets/TRsim/stages/:

- TRsim_room.glb: floor + walls + ceiling mesh.

- TRsim.ply: export the room, as well as the box.

- TRsim.house: .house file describing the room geometry, and the box location.

Within the folder data/scene_datasets/TRsim/objects/:

- TRsim_box.glb: box mesh.

And:

- TRsim_material_config.json: Acoustic properties file

## Step 1a: Readout human locations

Read out human locations from the Human1 SoundCAM dataset, save the locxyz per N (to later be able to have the location based on the filename). (/home/student/ss4/soundcam/examples.ipynb)

## Step 2: Modify the _semantic.ply + .house

Modify the .ply file to change the last 8 vertices (box) to different locations, as well as adding the right material "object_id". Save in the format "N_semantic.ply". 

Also, change the box xyz locations in the .house file.

## Step 3: Run SoundSpaces 2.0

Load the file, rename to "treated_room_object_semantic.ply", run simulation, and save as "N.wav".


--------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import numpy as np
import os

from plyfile import PlyData, PlyElement

In [2]:
BASE_DIR = "/home/student/ss"
SIM_DIR = os.path.join(BASE_DIR, "data/scene_datasets/TRsim")
STAGES_DIR = os.path.join(BASE_DIR, SIM_DIR, "stages")

### Step 1a: Readout human locations

In [3]:
CENTROID_PATH = "/home/student/ss4/soundcam-Dataset/TreatedRoom_preprocessed/Human1"
centroids = np.load(os.path.join(CENTROID_PATH, "centroid.npy"))
centroids = centroids/1000 # go from (mm) to (m)
print(centroids.shape) # (1000, 2), so thousand human xy positions, in (mm)

(1000, 2)


### Step 2: Modify the _semantic.ply + .house

In [4]:
def create_empty_ply(base_ply):
    base_verts = np.array(base_ply['vertex'].data) # (40,), where the first 33 correspond to the room, the last 8 to the box

    verts_dtype = [
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
        ('object_id', 'i4')] # remove the alpha, add the object_id
    
    new_verts = np.empty(len(base_verts), dtype=verts_dtype)

    new_verts['x'] = base_verts['x'] # last 8 will be overwritten
    new_verts['y'] = base_verts['y'] # last 8 will be overwritten
    new_verts['z'] = base_verts['z'] # does not change between measurement locations (vertex order does not change)
    new_verts['red'] = base_verts['red'] # same
    new_verts['green'] = base_verts['green'] # same
    new_verts['blue'] = base_verts['blue'] # same

    object_center_x = new_verts['x'][-1] - new_verts['x'][-8]
    object_center_y = new_verts['y'][-1] - new_verts['y'][-8]
    object_center = [object_center_x, object_center_y]

    min_wall_x = np.sort(new_verts['x'])[-13] # take the second min/max value, as this is the inner vertex of the wall
    min_wall_y = np.sort(new_verts['y'])[-13]
    max_wall_x = np.sort(new_verts['x'])[12]
    max_wall_y = np.sort(new_verts['y'])[12]
    minmax = [min_wall_x, min_wall_y, max_wall_x, max_wall_y]

    for v in range(len(new_verts)): # set all materials, does not change between measurement locations (vertex order does not change)
        if new_verts['red'][v] == 254:
            new_verts['object_id'][v] = 1 # wall material
        elif new_verts['green'][v] == 254:
            new_verts['object_id'][v] = 0 # floor material
        elif new_verts['blue'][v] == 254:
            new_verts['object_id'][v] = 2 # ceiling material
        else:
            new_verts['object_id'][v] = 3 # object material        
        
    faces = base_ply['face'] # unchanged, just sets which vertice ids correspond to which face (and vertex order does not change)
    
    return new_verts, faces, object_center, minmax

def modify_xyz_ply(new_verts, centroid, object_center, minmax):
    delta = centroid - object_center

    mod_verts = new_verts.copy()

    for v in range(8): # overwrite the last 8 vertex locations to match the new locations
        mod_verts['x'][-v-1] = new_verts['x'][-v-1] + delta[0]
        mod_verts['y'][-v-1] = new_verts['y'][-v-1] + delta[1]
    
    assert mod_verts['x'].all() <= minmax[0] and mod_verts['x'].all() >= minmax[2]
    assert mod_verts['y'].all() <= minmax[1] and mod_verts['y'].all() >= minmax[3]
    
    return mod_verts

def save_new_ply(mod_verts, faces, centroid_id, filename='TRsim_', mode='binary'):
    if mode == 'binary':
        ply = PlyData([PlyElement.describe(mod_verts, 'vertex'), faces], byte_order='<')
        ply.write(os.path.join(STAGES_DIR, filename + str(centroid_id) + "_semantic.ply"))
        # print(os.path.join(STAGES_DIR, filename + str(centroid_id) + "_semantic.ply"))
    elif mode == 'ascii': 
        ply = PlyData([PlyElement.describe(mod_verts, 'vertex'), faces], text=True)
        ply.write(os.path.join(STAGES_DIR, filename + str(centroid_id) + "_ascii.ply"))
    else:
        raise Exception("Please choose the mode to be either 'binary' or 'ascii'.")

In [5]:
def modify_O_house(base_house_path, centroid):
    with open(base_house_path, 'r') as f:
        lines = f.readlines()

    parts = lines[-1].split() # modify only the last O line
    parts[4] = str(centroid[0])
    parts[5] = str(centroid[1])
    lines[-1] = " ".join(parts) + "\n"

    return lines

def save_new_house(mod_house, centroid_id, filename='TRsim_'):
    out_path = os.path.join(STAGES_DIR, filename + str(centroid_id) + ".house")
    
    with open(out_path, 'w') as f:
        f.writelines(mod_house)

In [6]:
base_ply = PlyData.read(os.path.join(STAGES_DIR, "base", "TRsim.ply"))
new_verts, faces, object_center, minmax = create_empty_ply(base_ply)
print("Wall borders", minmax) # used for assertion for the new coords to be within the room

base_house_path = os.path.join(STAGES_DIR, "base", "TRsim.house")

for centroid_id in range(1000):
    mod_verts = modify_xyz_ply(new_verts, centroids[centroid_id], object_center, minmax)
    save_new_ply(mod_verts, faces, centroid_id)

    # O line: object_index region_index category_index px py pz  a0x a0y a0z  a1x a1y a1z  r0 r1 r2 0 0 0 0 0 0 0 0
    # where px py pz is the object bbox center, i.e. the centroid location
    mod_house = modify_O_house(base_house_path, centroids[centroid_id])
    save_new_house(mod_house, centroid_id)

Wall borders [3.7963, 1.3307, -1.3551, -4.3137]


### Step 3: Run SoundSpaces 2.0

Run Docker simsetup container with: 

(maybe: '''docker rm simsetup''')

'''docker run -it --gpus all --name simsetup -v ~/ss/examples:/sound-spaces/examples -v ~/ss/data:/sound-spaces/data soundspaces:U20cudaglheadless /bin/bash'''

'''cd sound-spaces'''

'''python examples/TRsim.py'''